### 確認 GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-11795d44-d721-10ac-9e9a-909204e64a2d)


### Mount 雲端硬碟

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Fine-tuning on your own

In [3]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/

/content/drive/My Drive/Colab Notebooks


### 下載 DRCD 資料集

In [4]:
!mkdir /content/drive/My\ Drive/Colab\ Notebooks/drcd
%cd /content/drive/My\ Drive/Colab\ Notebooks/drcd

mkdir: cannot create directory ‘/content/drive/My Drive/Colab Notebooks/drcd’: File exists
/content/drive/My Drive/Colab Notebooks/drcd


In [5]:
!wget https://github.com/DRCKnowledgeTeam/DRCD/blob/master/DRCD_training.json?raw=True -O train.json
!wget https://github.com/DRCKnowledgeTeam/DRCD/blob/master/DRCD_dev.json?raw=True -O dev.json

--2022-02-22 07:32:31--  https://github.com/DRCKnowledgeTeam/DRCD/blob/master/DRCD_training.json?raw=True
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/DRCKnowledgeTeam/DRCD/raw/master/DRCD_training.json [following]
--2022-02-22 07:32:31--  https://github.com/DRCKnowledgeTeam/DRCD/raw/master/DRCD_training.json
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DRCKnowledgeTeam/DRCD/master/DRCD_training.json [following]
--2022-02-22 07:32:32--  https://raw.githubusercontent.com/DRCKnowledgeTeam/DRCD/master/DRCD_training.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HT

### DRCD 資料格式
台達閱讀理解資料集 Delta Reading Comprehension Dataset (DRCD) 屬於通用領域繁體中文機器閱讀理解資料集。
本資料集期望成為適用於遷移學習之標準中文閱讀理解資料集。
本資料集從2,108篇維基條目中整理出10,014篇段落，並從段落中標註出30,000多個問題


For more information please refer to Paper: https://arxiv.org/abs/1806.00920

### Data format 資料格式

- version : <String> 資料集版本
- data : <Array>
  - title : <String> : 文章標題
  - id : <String> : 文章編號
  - paragraphs : <Array>
    - id : <String> : 文章編號_段落編號
    - context : <String> : 段落內容
    - qas : <Array>
      - question : <String> : 問題內容
      - id :<String> : 文章編號_段落編號_問題編號
      - answers : <Arrays>
        - answer_start : <int> text在文中位置
        - id : <String> : "1"表示為人工標註的答案，"2"以上為人工答題的答案
        - text : <string> : 答案內容

**References:**

- [DRCD](https://github.com/DRCKnowledgeTeam/DRCD)


In [6]:
import json
from pprint import pprint
with open('train.json') as file:
  train_data = json.load(file)
for ele in train_data['data']:
  pprint(ele)
  break

{'id': '1001',
 'paragraphs': [{'context': '2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。',
                 'id': '1001-10',
                 'qas': [{'answers': [{'answer_start': 84,
                                       'id': '1',
                                       'text': '10秒鐘'}],
                          'id': '1001-10-1',
                          'question': '廣州的快速公交運輸系統每多久就會有一輛巴士？'},
                         {'answers': [{'answer_start': 256,
                                       'id': '1',
                                       'text': '2007年1月16日'}],

### 載入相關套件

In [7]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 14.0 MB/s 
     |████████████████████████████████| 6.8 MB 49.3 MB/s 
     |████████████████████████████████| 895 kB 70.1 MB/s 
     |████████████████████████████████| 596 kB 52.8 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [8]:
import json
from pathlib import Path

import torch
from torch.utils.data import DataLoader
from transformers import (AdamW, BertForQuestionAnswering, BertTokenizerFast,
                          Trainer, TrainingArguments)

### 讀取資料（整理出資料中的 context、question、answer）

In [9]:
def read_drcd(path):
    path = Path(path)
    with open(path, 'rb') as f:
        drcd_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in drcd_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_drcd('./train.json')
val_contexts, val_questions, val_answers = read_drcd('./dev.json')

In [10]:
print(train_contexts[0])
print(train_questions[0])
print(train_answers[0])

2010年引進的廣州快速公交運輸系統，屬世界第二大快速公交系統，日常載客量可達100萬人次，高峰時期每小時單向客流高達26900人次，僅次於波哥大的快速交通系統，平均每10秒鐘就有一輛巴士，每輛巴士單向行駛350小時。包括橋樑在內的站台是世界最長的州快速公交運輸系統站台，長達260米。目前廣州市區的計程車和公共汽車主要使用液化石油氣作燃料，部分公共汽車更使用油電、氣電混合動力技術。2012年底開始投放液化天然氣燃料的公共汽車，2014年6月開始投放液化天然氣插電式混合動力公共汽車，以取代液化石油氣公共汽車。2007年1月16日，廣州市政府全面禁止在市區內駕駛摩托車。違反禁令的機動車將會予以沒收。廣州市交通局聲稱禁令的施行，使得交通擁擠問題和車禍大幅減少。廣州白雲國際機場位於白雲區與花都區交界，2004年8月5日正式投入運營，屬中國交通情況第二繁忙的機場。該機場取代了原先位於市中心的無法滿足日益增長航空需求的舊機場。目前機場有三條飛機跑道，成為國內第三個擁有三跑道的民航機場。比鄰近的香港國際機場第三跑道預計的2023年落成早8年。
廣州的快速公交運輸系統每多久就會有一輛巴士？
{'id': '1', 'text': '10秒鐘', 'answer_start': 84}


### 新增 answer 的結束位置

In [11]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text) # Find end character index of answer in context
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [12]:
print(train_answers[0])

{'id': '1', 'text': '10秒鐘', 'answer_start': 84, 'answer_end': 88}


### Tokenize（將資料轉換成 token id 、tpye_id 與 attention_mask）

In [13]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

In [14]:
print(train_encodings[0].ids)
print(tokenizer.decode(train_encodings[0].ids))

print(train_encodings[0].type_ids)
print(train_encodings[0].attention_mask)

[101, 8166, 2399, 2471, 6868, 4638, 2451, 2336, 2571, 6862, 1062, 769, 6880, 6745, 5143, 5186, 8024, 2253, 686, 4518, 5018, 753, 1920, 2571, 6862, 1062, 769, 5143, 5186, 8024, 3189, 2382, 6734, 2145, 7030, 1377, 6888, 8135, 5857, 782, 3613, 8024, 7770, 2292, 3229, 3309, 3680, 2207, 3229, 1606, 1403, 2145, 3837, 7770, 6888, 11023, 8279, 782, 3613, 8024, 1006, 3613, 3176, 3797, 1520, 1920, 4638, 2571, 6862, 769, 6858, 5143, 5186, 8024, 2398, 1772, 3680, 8108, 4907, 7132, 2218, 3300, 671, 6739, 2349, 1894, 8024, 3680, 6739, 2349, 1894, 1606, 1403, 6121, 7691, 8612, 2207, 3229, 511, 1259, 2886, 3578, 3558, 1762, 1058, 4638, 4991, 1378, 3221, 686, 4518, 3297, 7269, 4638, 2336, 2571, 6862, 1062, 769, 6880, 6745, 5143, 5186, 4991, 1378, 8024, 7269, 6888, 9044, 5101, 511, 4680, 1184, 2451, 2336, 2356, 1281, 4638, 6243, 4923, 6722, 1469, 1062, 1066, 3749, 6722, 712, 6206, 886, 4500, 3890, 1265, 4767, 3779, 3706, 868, 4234, 3160, 8024, 6956, 1146, 1062, 1066, 3749, 6722, 3291, 886, 4500, 3779, 7

### 經過 Tokenize 轉換後，重新計算 context 中 answer 的開始與結束位置

In [15]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

### 定義 Dataset，並轉換成 tensor 格式

In [16]:
class DrcdDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = DrcdDataset(train_encodings)
val_dataset = DrcdDataset(val_encodings)

In [17]:
print(train_dataset[0])

{'input_ids': tensor([  101,  8166,  2399,  2471,  6868,  4638,  2451,  2336,  2571,  6862,
         1062,   769,  6880,  6745,  5143,  5186,  8024,  2253,   686,  4518,
         5018,   753,  1920,  2571,  6862,  1062,   769,  5143,  5186,  8024,
         3189,  2382,  6734,  2145,  7030,  1377,  6888,  8135,  5857,   782,
         3613,  8024,  7770,  2292,  3229,  3309,  3680,  2207,  3229,  1606,
         1403,  2145,  3837,  7770,  6888, 11023,  8279,   782,  3613,  8024,
         1006,  3613,  3176,  3797,  1520,  1920,  4638,  2571,  6862,   769,
         6858,  5143,  5186,  8024,  2398,  1772,  3680,  8108,  4907,  7132,
         2218,  3300,   671,  6739,  2349,  1894,  8024,  3680,  6739,  2349,
         1894,  1606,  1403,  6121,  7691,  8612,  2207,  3229,   511,  1259,
         2886,  3578,  3558,  1762,  1058,  4638,  4991,  1378,  3221,   686,
         4518,  3297,  7269,  4638,  2336,  2571,  6862,  1062,   769,  6880,
         6745,  5143,  5186,  4991,  1378,  8024, 

### Train

In [18]:
# 設定訓練參數
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
)

#使用 bert-base-chinese 預訓練模型
model = BertForQuestionAnswering.from_pretrained("bert-base-chinese")

#將參數與資料丟入 trainer 
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese a

In [19]:
#開始訓練 一個　epoch 大約50分鐘
%%time
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 26936
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3367


Step,Training Loss
500,1.735700
1000,1.028000
1500,0.905500
2000,0.804000
2500,0.765600
3000,0.693100


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

CPU times: user 13min 48s, sys: 50.5 s, total: 14min 39s
Wall time: 17min 17s


TrainOutput(global_step=3367, training_loss=0.9534401050972096, metrics={'train_runtime': 1037.0494, 'train_samples_per_second': 25.974, 'train_steps_per_second': 3.247, 'total_flos': 7038289439440896.0, 'train_loss': 0.9534401050972096, 'epoch': 1.0})

In [20]:
#儲存模型

trainer.save_model("./results")

Saving model checkpoint to ./results
Configuration saved in ./results/config.json
Model weights saved in ./results/pytorch_model.bin


### Inference

In [21]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/drcd

/content/drive/My Drive/Colab Notebooks/drcd


In [29]:
from transformers import BertTokenizer, BertConfig, BertForQuestionAnswering
import torch
import textwrap

wrapper = textwrap.TextWrapper(width=80) 

tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
config = BertConfig.from_pretrained("./results/config.json") 
model = BertForQuestionAnswering.from_pretrained("./results/pytorch_model.bin", config=config)

context = "在前陣子公告將首度和迪士尼合作後，「新北歡樂耶誕城」的開城日一直是萬眾矚目的焦點，近日2020年的開園時間也確定將於2020年11月13日起至2021年1月3日，持續52天點亮新北之夜。新北市觀旅局透露，今年以「迪士尼經典童話」為主題，將重現《木偶奇遇記》、《愛麗絲夢遊仙境》、《小飛俠》、《小飛象》與 「迪士尼公主系列」等肖像之經典場景。此外，四大燈區包含主題燈飾、遊樂設施、藝術裝置等，共高達48座，創下歷年來最多，帶給民眾更豐富多元的視覺饗宴。"
questions = ["耶誕城首度跟誰合作?", "耶誕城會持續幾天?", "耶誕城今年是什麼主題?"]

print("Context: " + wrapper.fill(context) + '\n')
for question in questions:
    inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
    # input_ids = inputs["input_ids"].tolist()[0]
    input_ids = inputs["input_ids"][0]
    # print(inputs)
    output = model(**inputs)
    print("*************************")
    # print(output['start_logits'])
    answer_start = torch.argmax(output['start_logits'])
    answer_end = torch.argmax(output['end_logits']) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}")
    # input()

loading file https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/36acdf4f3edf0a14ffb2b2c68ba47e93abd9448825202377ddb16dae8114fe07.accd894ff58c6ff7bd4f3072890776c14f4ea34fcc08e79cd88c2d157756dceb
loading file https://huggingface.co/bert-base-chinese/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-chinese/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-chinese/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/2dc6085404c55008ba7fc09ab7483ef3f0a4ca2496ccee0cdbf51c2b5d529dff.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading file https://huggingface.co/bert-base-chinese/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/7e23f4e1f58f867d672f84d9a459826e41cea3be6d0fe62502ddce9920f57e48.4495f7812b44ff0568ce7c4ff3fdbb2bac5eaf330440ffa30f46893bf

Context: 在前陣子公告將首度和迪士尼合作後，「新北歡樂耶誕城」的開城日一直是萬眾矚目的焦點，近日2020年的開園時間也確定將於2020年11月13日起至2021年1月3日
，持續52天點亮新北之夜。新北市觀旅局透露，今年以「迪士尼經典童話」為主題，將重現《木偶奇遇記》、《愛麗絲夢遊仙境》、《小飛俠》、《小飛象》與
「迪士尼公主系列」等肖像之經典場景。此外，四大燈區包含主題燈飾、遊樂設施、藝術裝置等，共高達48座，創下歷年來最多，帶給民眾更豐富多元的視覺饗宴。

*************************
Question: 耶誕城首度跟誰合作?
Answer: 迪 士 尼
*************************
Question: 耶誕城會持續幾天?
Answer: 52 天
*************************
Question: 耶誕城今年是什麼主題?
Answer: 迪 士 尼 經 典 童 話
